In [ ]:
#hide
#default_exp models

# Dual Store ICMR Prototype
With this version of the model, we make two kinds of modifications. 

## Theoretical Modifications
Theoretically, we make a major modification to the ICMR model to improve comparison with CMR dynamics.

In particular, we separate item and contextual features when representing each `experience`'s associated
memory `trace`. If item features or context don't exist or aren't relevant for specification of a particular
trace, that part of the trace representation will just be a zero vector.

This enables us to organize retrieval selectively. A feature `echo` associated with the current `context`
representation can be obtained by presenting `context` with a null `feature` representation as a `probe` to
`memory`, effectively performing the role of CMR's `Mcf`. Similarly, item features can be presented with null
context features as a memory probe to measure relevant contextual associations, operating similarly to CMR's
`Mfc`.

Besides this, we also make substantial modifications to our retrieval mechanism. Before, item activations
were computed by comparing the current contextual representation to each item's feature representation. Now,
we'll try an operation that hews better with the classical `echo` mechanism. We'll present `context` as a cue
to memory in the manner described above, and use the resulting trace activations to determine recall
competition. 

The way CMR and instance models compute similarity-based activation are quite similar, but a potentially
important distinction is that while CMR generates activations for every item that might be recalled, instance
models' echo mechanism measures activations for every stored trace. So if the same item has been presented
repeatedly, activation vectors at minimum will exhibit different lengths. In the current version of the
model, we allow traces associated with the same items to compete with one another in the activation function.
However, once an item is recalled, we still rule out repeated recall of the same item for the rest of the
experiment. This is expected to lead to similar recall dynamics as to how CMR organizes recall.

We also go ahead and include support for all parameters parallel to those specified in the Morton and Polyn
2016 paper. A key experiment down the line will be to explore whether manipulating these parameters impacts
model performance like they would in CMR.

## Operational Modifications
To support easier inspection of model states, we include several functions that either refactor operations
previously performed by larger superfunctions or that control model operation in novel ways.

The function `activations` collects the level of activation of each stored memory trace in the model in
response to a specified `probe`. `outcome_probabilities` alternatively computes the likelihood of retrieving
each encoded item, and can be computed either on the basis of the current state of context or with optional
`cue` or `activations` parameters. Finally, the `force_recall` function forces model to recall chosen item
and update context regardless of current model state. This is useful for testing particular hypotheses about
model dynamics.

In [ ]:
#exports
import math
import numpy as np
from scipy import spatial
from numpy.linalg import norm


class DualStorePrototype(object):
    """
    The context maintenance and retrieval model re-imagined as an exemplar model.

    Attributes:  
    - memory: array where rows/columns correspond to accumulated memory traces and feature dims respectively
    - context: vector reflecting a recency-weighted average of recently presented stimuli information
    - item_count: number of unique items relevant to model simulation
    - encoding_drift_rate: rate of context drift during item processing
    - start_drift_rate: amount of start-list context retrieved at start of recall
    - recall_drift_rate: rate of context drift during recall
    - shared_support: uniform amount of support items initially have for one another in recall competition
    - item_support: initial strength of the diagonal of Mcf
    - learning_rate: contribution of experimental relative to pre-experimental associations
    - primacy_scale: scaling of primacy gradient in learning rate on Mcf
    - primacy_decay: rate of decay of primacy gradient
    - semantic_scale: scaling of semantic association strengths
    - stop_probability_scale: scaling of the stop probability over output position
    - stop_probability_growth: rate of increase in stop probability
    - choice_sensitivity: sensitivity parameter of the Luce choice rule
    """

    def __init__(self, item_count, encoding_drift_rate, start_drift_rate, recall_drift_rate, shared_support,
                 item_support, learning_rate, primacy_scale, primacy_decay, stop_probability_scale,
                 stop_probability_growth, choice_sensitivity):
        """
        Start exemplar model with an initial set of pre-experimental associations in memory

        Args:  
            item_count: number of unique items relevant to model simulation encoding_drift_rate: rate of
            context drift during item processing start_drift_rate: amount of start-list context retrieved at
            start of recall recall_drift_rate: rate of context drift during recall shared_support: uniform
            support items initially have for one another in recall competition item_support: initial strength
            of the diagonal of Mcf learning_rate: contribution of experimental relative to pre-experimental
            associations primacy_scale: scaling of primacy gradient in learning rate on Mcf primacy_decay: rate
            of decay of primacy gradient semantic_scale: scaling of semantic association strengths
            stop_probability_scale: scaling of the stop probability over output position
            stop_probability_growth: rate of increase in stop probability choice_sensitivity: sensitivity
            parameter of the Luce choice rule
        """

        # store initial parameters
        self.item_count = item_count
        self.encoding_drift_rate = encoding_drift_rate
        self.start_drift_rate = start_drift_rate
        self.recall_drift_rate = recall_drift_rate
        self.shared_support = shared_support
        self.item_support = item_support
        self.learning_rate = learning_rate
        self.primacy_scale = primacy_scale
        self.primacy_decay = primacy_decay
        # self.semantic_scale = semantic_scale
        self.stop_probability_scale = stop_probability_scale
        self.stop_probability_growth = stop_probability_growth
        self.choice_sensitivity = choice_sensitivity

        # at the start of the list context is initialized with a state orthogonal to the pre-experimental
        # context associated with the set of items
        self.context = np.eye(1, item_count + 1).flatten()
        self.preretrieval_context = self.context
        self.state = 'encoding'
        self.recall = []

        # initialize memory
        # we now conceptualize it as a pairing of two stores Mfc and Mcf representing feature-to-context and
        # context-to-feature associations, respectively
        mfc = np.eye(item_count, item_count + 1, 1) * (1 - self.learning_rate)
        mcf = np.eye(item_count) * self.item_support
        mcf[np.logical_not(np.eye(item_count, dtype=bool))] = self.shared_support
        mcf = np.hstack((np.zeros((item_count, 1)), mcf))
        self.memory = np.hstack((mfc, mcf))

        # an additional item store and KDTree organizes response selection based on retrieved F
        self.items = np.eye(self.item_count, self.item_count + 1, 1)
        self.item_decision_rule = spatial.KDTree(self.items)

    def experience(self, experiences):
        """"
        Adds new trace(s) to model memory based on an experienced vector of item features and current context.
            """
        if len(np.shape(experiences)) == 1:
            experiences = [experiences]
        for experience in experiences:
            self.update_context(
                self.encoding_drift_rate, np.hstack((np.array(experience), np.zeros((self.item_count + 1)))))
            trace = np.hstack((experience, self.context))
            self.memory = np.vstack((self.memory, trace))

    def update_context(self, drift_rate, experience=None):
        """
        Updates contextual representation based on content of current experience or, if no experience is
        presented, on the content of initial context.
        """

        # first pre-experimental or initial context is retrieved
        if experience is not None:
            context_input = self.echo(experience)[self.item_count + 1:]
            context_input = context_input / norm(context_input)  # normalized to have length 1
        else:
            context_input = np.eye(1, self.item_count + 1).flatten()

        # updated context is sum of context and input, modulated by rho to have len 1 and some drift_rate
        rho = np.sqrt(1 + (drift_rate ** 2) * (((self.context * context_input) ** 2) - 1)) - (
                drift_rate * (self.context * context_input))
        self.context = (rho * self.context) + (drift_rate * context_input)

    def echo(self, probe):
        """
        A probe activates all traces in memory in parallel. The sum of these traces weighted by their
        activation is an `echo` summarizing the memory system's response to the probe.
        """
        return np.sum((self.memory.T * self.activations(probe)).T, axis=0)

    def compare_probes(self, first_probe, second_probe):
        """Compute the resemblance (cosine similarity) between the echoes associated with probes A and B."""
        echoes = self.echo(first_probe), self.echo(second_probe)
        return np.sum(echoes[0] * echoes[1]) / (norm(echoes[0]) * norm(echoes[1]))

    def activations(self, probe):
        """
        Presents a cue to memory system, activating all traces in memory in parallel. Each trace's `activation`
        is a cubed function of its `similarity` to the probe, weighted based on item position and whether probe
        contains contextual or item features.
        """
        # computes and cubes similarity value to find activation for each trace in memory
        activation = np.sum(self.memory * probe, axis=1) / (norm(self.memory, axis=1) * norm(probe))
        activation = activation ** self.choice_sensitivity

        # determining weighting based on whether probe contains item or contextual features mfc weightings -
        # scale by gamma for each experimental trace
        weighting = np.ones(len(self.memory))
        if np.sum(probe[:self.item_count + 1]) != 0:
            weighting[self.item_count:] = self.learning_rate

        # mcf weightings - scale by primacy/attention function based on position of experimental experiences
        if np.sum(probe[self.item_count + 1:]) != 0:
            weighting[self.item_count:] *= self.primacy_scale * np.exp(
                -self.primacy_decay * np.arange(len(self.memory) - self.item_count)) + 1

        # weights traces based on provided scalings
        activation *= weighting
        return activation  + np.finfo(float).eps

    def outcome_probabilities(self, activations=None, cues=None):
        """
        Computes recall probability of each item given current model state and optional activation pattern/cue.
        """

        # if no activations are provided, generate some from current context
        if activations is None:
            activations = self.activations(np.hstack((np.zeros(self.item_count + 1), self.context)))

        # temporarily update context to reflect cue(s)
        preretrieval_context = self.context
        if cues is not None:
            for cue in cues:
                self.update_context(cue, self.retrieval_drift_rate)

        outcome_probabilities = np.zeros((self.item_count + 1))
        outcome_probabilities[0] = min(self.stop_probability_scale * np.exp(
            len(self.recall) * self.stop_probability_growth), 1.0)

        if outcome_probabilities[0] < 1:
            for trace_index, trace in enumerate(self.memory.tolist()):
                j = self.item_decision_rule.query(trace[:self.item_count + 1])[1]
                if j in self.recall:
                    continue
                outcome_probabilities[j + 1] += activations[trace_index]
            outcome_probabilities[1:] *= (1 - outcome_probabilities[0]) / np.sum(outcome_probabilities[1:])

        self.context = preretrieval_context
        return outcome_probabilities

    def force_recall(self, choice=None):
        """
        Forces model to recall chosen item and update context regardless of current model state.

        Here, recall items are 1-indexed, with a choice of 0 indicating a choice to end retrieval and return to
        preretrieval context.
        """
        if self.state == 'encoding':
            self.recall = []
            self.preretrieval_context = self.context
            self.update_context(self.start_drift_rate)
            self.state = 'retrieving'
            
        if choice is None:
            pass
        elif choice == 0:
            self.state = 'encoding'
            self.context = self.preretrieval_context
        else:
            self.recall.append(choice - 1)
            self.update_context(self.recall_drift_rate,
                                np.hstack((self.items[choice - 1], np.zeros(self.item_count + 1))))
        return self.recall

    def free_recall(self, steps=None):
        """
        Simulates performance on a free recall task based on experienced items.

        We initialize context similar to eq. 16 from Morton & Polyn (2016), simulating end-of-list distraction
        and some amount of pre-list context reinstatement. This context is used as a retrieval cue (probe) to
        attempt retrieval of a studied item, generating an associated memory echo.

        At each recall attempt, we also calculate a probability of stopping recall as a function of output
        position according to eq. 18 of Morton & Polyn (2016). The probability of recalling a given item
        conditioned on not stopping recall is defined on the basis of the item's similarity to the current
        contextual representation according to a formula similar to eq 19 of Morton and Polyn (2016).
        """
        # some amount of the pre-list context is reinstated before initiating recall
        if self.state == 'encoding':
            self.recall = []
            self.preretrieval_context = self.context
            self.update_context(self.start_drift_rate)
            self.state = 'retrieving'

        # number of items to retrieve is infinite if steps is unspecified
        if steps is None:
            steps = math.inf
        steps = len(self.recall) + steps

        # at each recall attempt
        while len(self.recall) < steps:

            # the current state of context is used as a retrieval cue to attempt recall of a studied item
            activations = self.activations(np.hstack((np.zeros(self.item_count + 1), self.context)))

            # compute outcome probabilities and make choice based on distribution
            outcome_probabilities = self.outcome_probabilities(activations)
            choice = np.random.choice(len(outcome_probabilities), p=outcome_probabilities)

            # resolve and maybe store outcome
            # we stop recall if no choice is made
            if not choice:
                self.state = 'encoding'
                self.context = self.preretrieval_context
                break
            self.recall.append(choice - 1)
            self.update_context(self.recall_drift_rate,
                                np.hstack((self.items[choice - 1], np.zeros(self.item_count + 1))))

        return self.recall

## Miscellaneous Issues

### Missing Details
Some aspects that are not faithful to CMR as specified by Morton and Polyn (2016):
- We COULD set a minimal activation for each item to 10^-7 here but we don't yet.
- We could go ahead and add support for semantic associations, but we don't yet.

### How Should the DualStorePrototype Choose Responses Under Ambiguously Retrieved Traces?

I missed something when brainstorming this model. The traces I lay down for new items don't always contain an
exact matching feature representation for the item. For example, at model initialization, gamma modulates
item to context associations a bit (1-gamma instead of 1).

It would be highly irregular if I had a separate representation labelling each trace by its associated item.
Though I guess CMR does the same thing.

I could definitely modify the model to enforce stable feature representations. For example, I could take the
gamma modulation of pre-experimental activations out of the memory array like I've already done for
experimental items. But the problem still remains in a subtle way: I don't get to "mess" with feature
representations later, for example.

A nearest neighbor approach is possible, but which is the one I can best justify in the context of CMR and
what I've read of instance models?

#### Which Policies are Possible?

We can just apply a classification rule to choose when the recovered trace is ambiguous. We can have a
separate store of the decision associated with an item (e.g. a perfect representation of the item or just an
index stored for each trace). However, this feels like cheating, and when we start thinking about memory
failure/bias, this gets weird, too.

These might be our only options! 

I definitely like making selection happen for particular traces. I want my decision rule to depend on
particular traces. 

Why should the content of my Mfc vector be what decides what's stored though? The Mfc and Mcf are supposed to
be realized on the basis of co-occurrence of feature and contextual information in the same trace. Our two
store model isn't a combination of Mfc and Mcf information; it's a combination of feature and contextual
information; Mcf and Mfc mechanisms just emerge from their combination. So I do want my F store to be a
faithful store of encoded feature information.

I resolved in this model to go with a classification strategy: the nearest applicable item is selected for
retrieval. Further exploration of retrieval mechanisms might be worthwhile. Some further discussion in past
relevant papers below.

#### Hintzman, D. L. (1984). MINERVA 2: A simulation model of human memory. Behavior Research Methods, Instruments, & Computers, 16(2), 96-101.

"The ambiguous recall problem is that information retrieved from memory is sometimes only vaguely similar to
what was originally stored, or to any acceptable response. One way of getting around this difficulty is to
assume that there is a pattern-recognition system that knows the set of acceptable responses and that takes
the memory echo as input to be classified (e.g., Eich, 1982). But MINERVA 2 lends itself to a more elegant
solution. If the echo content is "normalized" into the -1 to +1 range and turned into a secondary probe, the
subsequent echo is more like an acceptable response than the original echo was. And if this process is
repeated, there is further improvement. Typically, only a few iterations of this procedure are necessary to
produce a virtually perfect copy of the information that was originally stored. In this way, MINERVA 2
appears able to "bootstrap" its way out of the ambiguous recall problem with no help from a memory system on
the outside."

There's never a perfect equality here, here, even after repeated probing. I can set some sort of threshold
for selecting a match after enough iterations, but if I'm doing that, why not pull a nearest-neighbors
selection approach? Classification is required if I'm doing decision-making.

#### Morton, N. W., & Polyn, S. M. (2016). A predictive framework for evaluating models of semantic organization in free recall. Journal of Memory and Language, 86, 119-140.

"At each recall attempt, the current state of context is used as a retrieval cue to attempt retrieval of a
studied item. First, the activation of each item a is determined according to... In order to avoid the
possibility of the model assigning a probability of 0 to any possible recall, we set a minimal activation for
each item of 10^-7.

At each recall attempt, we calculated the probability of stopping recall (in which case no item was recalled,
and search terminated). Probability of stopping recall varies as a function of output position j (where j = 0
for the first attempt), according to: P(stop, j) = ... where hs and hr are free parameters that determine the
scaling and rate of increase, respectively, of the exponential function. The stopping mechanism does not
interact with any model mechanism, and is simply intended to capture the average probability of stopping as a
function of output position. The probability P(i) of recalling a given item i is defined conditional on
recall not stopping at that position, and it varies with activation strength, according to P(i) = ... where s
is a sensitivity parameter that determines the contrast between well-supported and poorly supported items.
High values of s will cause a greater influence of differences in support, while low values will cause
relatively uniform probabilities of recalling each item.
"

So this is a classification scheme. We want to follow this, so we want the probability of recalling an item
to be a linear combination of the probability of recalling each relevant trace. There are tons of ways to
potentially realize this. The important thing is just picking one! In my mind, a nearest neighbor approach is
reasonably solid. It represents the likely outcome of more complex selection processes we can think of, but
it still makes room for behavior to depend on the content of the stored trace, which can be messed with in
lots of conceivable ways.